In [1]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt

# Genetic Algorithm for Job Shop Scheduling Problem (JSSP)


In [2]:
def parse_dataset(file_path, instance_name):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    start_idx = -1
    for idx, line in enumerate(lines):
        if line.strip() == f"instance {instance_name}": 
            start_idx = idx  #If the line matches "instance {instance_name}", record its position (start_idx).

            break 

    if start_idx == -1: #If the instance isn’t found (start_idx remains -1), raise an error.
        raise ValueError(f"Instance {instance_name} not found in the dataset file.")

    job_data = []
    for idx in range(start_idx + 1, len(lines)):
        line = lines[idx].strip()
        if line.startswith('+') or line == "":
            continue
        try:
            job_count, machine_count = map(int, line.split())
            count_line_idx = idx
            break
        except ValueError:
            continue
    else:
        raise ValueError(f"Failed to find job and machine counts for instance {instance_name}.")

    for line in lines[count_line_idx + 1:]:
        line = line.strip()
        if line.startswith('+') or line == "":
            break
        tasks = list(map(int, line.split()))
        job = [(tasks[i], tasks[i + 1]) for i in range(0, len(tasks), 2)]
        job_data.append(job)  ##Look for the line that specifies the number of jobs  and machines.


    if len(job_data) != job_count:
        raise ValueError(f"Mismatch between declared job count ({job_count}) and parsed jobs ({len(job_data)}).")

    return job_data, machine_count


In [3]:
''''
# Define the data for the Job Shop Scheduling Problem (JSSP)
jobs_data = [
    [(0, 3), (1, 2), (2, 2)],  # Job 0
    [(0, 2), (2, 1), (1, 4)],  # Job 1
    [(1, 4), (2, 3)],          # Job 2
    [(0, 2), (1, 3), (2, 1)],  # Job 3
    [(1, 5), (2, 2), (0, 4)]   # Job 4
]

machines_count = 1 + max(task[0] for job in jobs_data for task in job)
all_machines = range(machines_count)
horizon = sum(task[1] for job in jobs_data for task in job) 

'''

"'\n# Define the data for the Job Shop Scheduling Problem (JSSP)\njobs_data = [\n    [(0, 3), (1, 2), (2, 2)],  # Job 0\n    [(0, 2), (2, 1), (1, 4)],  # Job 1\n    [(1, 4), (2, 3)],          # Job 2\n    [(0, 2), (1, 3), (2, 1)],  # Job 3\n    [(1, 5), (2, 2), (0, 4)]   # Job 4\n]\n\nmachines_count = 1 + max(task[0] for job in jobs_data for task in job)\nall_machines = range(machines_count)\nhorizon = sum(task[1] for job in jobs_data for task in job) \n\n"

In [4]:
def is_valid_chromosome(chromosome, jobs_data):
    task_tracker = {job_id: 0 for job_id in range(len(jobs_data))}
    for job_id, (machine, duration) in chromosome:
        if task_tracker[job_id] < len(jobs_data[job_id]) and jobs_data[job_id][task_tracker[job_id]] == (machine, duration):
            task_tracker[job_id] += 1
        else:
            return False
    return all(task_tracker[job_id] == len(jobs_data[job_id]) for job_id in range(len(jobs_data)))


In [5]:
def repair_chromosome(chromosome, jobs_data):
    task_tracker = {job_id: 0 for job_id in range(len(jobs_data))}
    repaired_chromosome = []

    for job_id, (machine, duration) in chromosome:
        # Ensure the task tracker does not exceed the task list length
        if task_tracker[job_id] < len(jobs_data[job_id]):
            expected_task = jobs_data[job_id][task_tracker[job_id]]
            if (machine, duration) == expected_task:
                repaired_chromosome.append((job_id, (machine, duration)))
                task_tracker[job_id] += 1

    # Fill in any missing tasks in the correct order
    for job_id in range(len(jobs_data)):
        while task_tracker[job_id] < len(jobs_data[job_id]):
            task = jobs_data[job_id][task_tracker[job_id]]
            repaired_chromosome.append((job_id, task))
            task_tracker[job_id] += 1

    return repaired_chromosome

In [6]:
def validate_and_repair_population(population, jobs_data):
    return [repair_chromosome(chromosome, jobs_data) for chromosome in population]

In [7]:
def generate_population(jobs_data, population_size):
    population = []
    for i in range(population_size):
        chromosome = []
        for job_id, job in enumerate(jobs_data):
            for task in job:
                chromosome.append((job_id, task))
        if i < population_size // 2:
            random.shuffle(chromosome)
        # Repair the chromosome to ensure validity
        chromosome = repair_chromosome(chromosome, jobs_data)
        population.append(chromosome)
    return population

In [8]:
# Fitness function: Calculate makespan
def calculate_fitness(chromosome, machines_count, jobs_data):
    machine_times = [0] * machines_count
    job_times = [0] * len(jobs_data)

    for job_id, (machine, duration) in chromosome:
        start_time = max(machine_times[machine], job_times[job_id])
        end_time = start_time + duration
        machine_times[machine] = end_time
        job_times[job_id] = end_time

    return max(machine_times)


In [9]:
# Selection: Tournament selection
def tournament_selection(population, fitness, k=3):
    selected = random.sample(list(zip(population, fitness)), k)
    return min(selected, key=lambda x: x[1])[0]

In [10]:
# Crossover: One-point crossover
def two_point_crossover(parent1, parent2):
    # Choose two random crossover points
    point1, point2 = sorted(random.sample(range(len(parent1)), 2))

    # Create children by swapping the segments between point1 and point2
    child1 = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
    child2 = parent2[:point1] + parent1[point1:point2] + parent2[point2:]

    return child1, child2


In [11]:
def uniform_crossover(parent1, parent2):
    child1, child2 = [], []
    for gene1, gene2 in zip(parent1, parent2):
        if random.random() < 0.5:
            child1.append(gene1)
            child2.append(gene2)
        else:
            child1.append(gene2)
            child2.append(gene1)

    return child1, child2


In [12]:
# Mutation: Always One Mutation per Chromosome ,To prevent getting stuck with bad solutions.
def always_one_mutation(chromosome):
    if len(chromosome) <= 1:
        return chromosome
    mutation_index = random.randint(0, len(chromosome) - 1)
    swap_index = random.randint(0, len(chromosome) - 1)
    chromosome[mutation_index], chromosome[swap_index] = chromosome[swap_index], chromosome[mutation_index]
    return chromosome



In [13]:
# Mutation: Each Gene Mutates Independently
def independent_gene_mutation(chromosome, mutation_rate):
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            swap_index = random.randint(0, len(chromosome) - 1)
            chromosome[i], chromosome[swap_index] = chromosome[swap_index], chromosome[i]
    return chromosome


In [14]:
# Genetic Algorithm implementation
def genetic_algorithm(jobs_data, population_size=50, generations=100, crossover_rate=0.8, mutation_rate=0.1, elite_count=2):
    machines_count = len(set(machine for job in jobs_data for machine, _ in job))
    population = generate_population(jobs_data, population_size)
    population = validate_and_repair_population(population, jobs_data)
    best_solution = None
    best_fitness = float('inf')
    fitness_history = []

    for generation in range(generations):
        fitness = [calculate_fitness(chromosome, machines_count, jobs_data) for chromosome in population]
        sorted_population = [x for _, x in sorted(zip(fitness, population), key=lambda pair: pair[0])]
        sorted_fitness = sorted(fitness)

        elites = sorted_population[:elite_count]
        new_population = elites[:]

        while len(new_population) < population_size:
            parent1 = tournament_selection(population, fitness)
            parent2 = tournament_selection(population, fitness)
            if random.random() < crossover_rate:
                if random.random() < 0.5:
                    child1, child2 = two_point_crossover(parent1, parent2)
                else:
                    child1, child2 = uniform_crossover(parent1, parent2)
            else:
                child1, child2 = parent1, parent2

            child1 = repair_chromosome(child1, jobs_data)
            child2 = repair_chromosome(child2, jobs_data)

            if random.random() < mutation_rate:
                child1 = always_one_mutation(child1)
                child2 = always_one_mutation(child2)

            child1 = repair_chromosome(child1, jobs_data)
            child2 = repair_chromosome(child2, jobs_data)

            new_population.append(child1)
            if len(new_population) < population_size:
                new_population.append(child2)

        population = new_population

        best_gen_fitness = sorted_fitness[0]
        best_gen_solution = sorted_population[0]

        if best_gen_fitness < best_fitness:
            best_fitness = best_gen_fitness
            best_solution = best_gen_solution

        fitness_history.append(best_fitness)

    return best_solution, best_fitness, fitness_history



In [15]:
# Simulated Annealing implementation
def simulated_annealing(jobs_data, initial_solution, initial_temperature, cooling_rate, max_iterations):
    machines_count = len(set(machine for job in jobs_data for machine, _ in job))  # Total machines

    # Explore neighbors and improve
    def neighbor_solution(current_solution):
        neighbor = current_solution[:]
        i, j = random.sample(range(len(neighbor)), 2)
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]  # Random swap
        return neighbor

    def repair_neighbor_solution(neighbor, jobs_data):
        return repair_chromosome(neighbor, jobs_data)

    current_solution = initial_solution
    current_fitness = calculate_fitness(current_solution, machines_count, jobs_data)
    best_solution = current_solution
    best_fitness = current_fitness
    temperature = initial_temperature

    fitness_history = [current_fitness]

    for iteration in range(max_iterations):
        # Generate and repair neighbor solution
        new_solution = neighbor_solution(current_solution)
        new_solution = repair_neighbor_solution(new_solution, jobs_data)

        new_fitness = calculate_fitness(new_solution, machines_count, jobs_data)

        # Acceptance probability
        if new_fitness < current_fitness or random.random() < math.exp(-(new_fitness - current_fitness) / temperature):
            current_solution = new_solution
            current_fitness = new_fitness

            if new_fitness < best_fitness:
                best_solution = new_solution
                best_fitness = new_fitness

        # Cool down the temperature
        temperature *= cooling_rate
        fitness_history.append(best_fitness)

        if temperature < 1e-6:
            break

    return best_solution, best_fitness, fitness_history


In [ ]:
# Main code
file_path = 'jobshop1.txt'
instance_name = 'abz9'

try:
    # Parse dataset
    jobs_data, machines_count = parse_dataset(file_path, instance_name)
    print(f"Parsed Jobs Data: {jobs_data}")
    print(f"Machines Count: {machines_count}")

    # Define GA parameter combinations
    ga_parameter_combinations = [
        {"population_size": 50, "mutation_rate": 0.1, "crossover_rate": 0.8, "elitism": 2},
        {"population_size": 100, "mutation_rate": 0.05, "crossover_rate": 0.7, "elitism": 2},
        {"population_size": 50, "mutation_rate": 0.2, "crossover_rate": 0.6, "elitism": 1},
        {"population_size": 200, "mutation_rate": 0.2, "crossover_rate": 0.9, "elitism": 3},
        {"population_size": 75, "mutation_rate": 0.1, "crossover_rate": 0.8, "elitism": 2},
        {"population_size": 50, "mutation_rate": 0.05, "crossover_rate": 0.9, "elitism": 1},
    ]

    # Store GA results
    ga_results = []

    # Run Genetic Algorithm for each parameter combination
    for params in ga_parameter_combinations:
        print(f"\nRunning GA with parameters: {params}")
        ga_best_solution, ga_best_fitness, ga_fitness_history = genetic_algorithm(
            jobs_data, params["population_size"], 100, params["crossover_rate"],
            params["mutation_rate"], params["elitism"]
        )

        # Save results
        ga_results.append({
            "parameters": params,
            "best_fitness": ga_best_fitness,
            "best_solution": ga_best_solution,
            "fitness_history": ga_fitness_history,
        })

        # Print results for GA
        print(f"GA Results - Parameters: {params}")
        print(f"Best Fitness (Makespan): {ga_best_fitness}")
        print(f"Best Solution: {ga_best_solution}")

        # Plot fitness evolution
        plt.figure()
        plt.plot(ga_fitness_history, label=f"GA - Pop: {params['population_size']}, Mut: {params['mutation_rate']}, X: {params['crossover_rate']}, E: {params['elitism']}")
        plt.xlabel('Generation')
        plt.ylabel('Best Fitness (Makespan)')
        plt.title(f"Fitness Evolution (GA - Pop: {params['population_size']}, Mut: {params['mutation_rate']}, X: {params['crossover_rate']}, E: {params['elitism']})")
        plt.legend()
        plt.savefig(f"fitness_evolution_ga_pop{params['population_size']}_mut{params['mutation_rate']}_xover{params['crossover_rate']}_elitism{params['elitism']}.png")
        plt.close()

    # Define SA parameter combinations
    sa_parameter_combinations = [
        {"initial_temperature": 1000, "cooling_rate": 0.99, "max_iterations": 1000},
        {"initial_temperature": 500, "cooling_rate": 0.95, "max_iterations": 1000},
        {"initial_temperature": 1500, "cooling_rate": 0.98, "max_iterations": 1500},
        {"initial_temperature": 800, "cooling_rate": 0.97, "max_iterations": 1200},
        {"initial_temperature": 1200, "cooling_rate": 0.96, "max_iterations": 1500},
        {"initial_temperature": 700, "cooling_rate": 0.94, "max_iterations": 1100},
    ]

    # Store SA results
    sa_results = []

    # Run Simulated Annealing for each parameter combination
    for params in sa_parameter_combinations:
        print(f"\nRunning SA with parameters: {params}")
    #one solution , first only solution
        sa_initial_solution = generate_population(jobs_data, 1)[0]
        sa_best_solution, sa_best_fitness, sa_fitness_history = simulated_annealing(
            jobs_data, sa_initial_solution, params["initial_temperature"], params["cooling_rate"], params["max_iterations"]
        )

        # Save results
        sa_results.append({
            "parameters": params,
            "best_fitness": sa_best_fitness,
            "best_solution": sa_best_solution,
            "fitness_history": sa_fitness_history,
        })

        # Print results for SA
        print(f"SA Results - Parameters: {params}")
        print(f"Best Fitness (Makespan): {sa_best_fitness}")
        print(f"Best Solution: {sa_best_solution}")

        # Plot fitness evolution
        plt.figure()
        plt.plot(sa_fitness_history, label=f"SA - Temp: {params['initial_temperature']}, Cool: {params['cooling_rate']}, Iter: {params['max_iterations']}")
        plt.xlabel('Iteration')
        plt.ylabel('Best Fitness (Makespan)')
        plt.title(f"Fitness Evolution (SA - Temp: {params['initial_temperature']}, Cool: {params['cooling_rate']}, Iter: {params['max_iterations']})")
        plt.legend()
        plt.savefig(f"fitness_evolution_sa_temp{params['initial_temperature']}_cool{params['cooling_rate']}_iter{params['max_iterations']}.png")
        plt.close()

except Exception as e:
    print(f"Error: {e}")

Parsed Jobs Data: [[(2, 24), (3, 12), (9, 17), (4, 27), (0, 21), (6, 25), (8, 27), (7, 26), (1, 30), (5, 31), (11, 18), (14, 16), (13, 39), (10, 19), (12, 26)], [(6, 30), (3, 15), (12, 20), (11, 19), (1, 24), (13, 15), (10, 28), (2, 36), (5, 26), (7, 15), (0, 11), (8, 23), (14, 20), (9, 26), (4, 28)], [(6, 35), (0, 22), (13, 23), (7, 32), (2, 20), (3, 12), (12, 19), (10, 23), (9, 17), (1, 14), (5, 16), (11, 29), (8, 16), (4, 22), (14, 22)], [(9, 20), (6, 29), (1, 19), (7, 14), (12, 33), (4, 30), (0, 32), (5, 21), (11, 29), (10, 24), (14, 25), (2, 29), (3, 13), (8, 20), (13, 18)], [(11, 23), (13, 20), (1, 28), (6, 32), (7, 16), (5, 18), (8, 24), (9, 23), (3, 24), (10, 34), (2, 24), (0, 24), (14, 28), (12, 15), (4, 18)], [(8, 24), (11, 19), (14, 21), (1, 33), (7, 34), (6, 35), (5, 40), (10, 36), (3, 23), (2, 26), (4, 15), (9, 28), (13, 38), (12, 13), (0, 25)], [(13, 27), (3, 30), (6, 21), (8, 19), (12, 12), (4, 27), (2, 39), (9, 13), (14, 12), (5, 36), (10, 21), (11, 17), (1, 29), (0, 17